In [8]:
# Initialize Otter
import otter
grader = otter.Notebook("hw5.ipynb")

---

<h1><center>SDSE Homework 5 <br><br> Text Classification with Naive Bayes </center></h1>

---

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from hashutils import *

In this homework we will apply the technique of Naive Bayes classification to the problem of categorizing text-based documents. The dataset is a selection of posts from scikit-learn's ['20 newsgroups' dataset](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset), which contains some 18000 newsgroup posts in 20 different categories, such as politics, autos, electronics, atheism, and hockey. For simplicity, we will focus on just two of the categories: computer graphics and motorcycles.

In [3]:
with open('hw5_text.pickle','rb') as file:
    Xtrain, ytrain, Xval, yval, categories, vocabulary = pickle.load(file)

# 1. Define constants

Define the following variables in terms of quantities loaded from the pickle file. 
+ `N` ... number of documents in the training corpus
+ `K` ... number of document categories
+ `D` ... number of words in the vocabulary

In [25]:
N = Xtrain.shape[0]
K = len(categories)
D = len(vocabulary)

In [26]:
grader.check("q1")

q1 results: All test cases passed!

# 2. Find the number of training documents for each category

Create a Python dictionary called `docs_per_category` that stores the number of documents in the training data under each category.
The keys of `docs_per_category` are the two categories (`comp.graphics` and `rec.motorcycles`). The value for each key is an integer corresponding to the number of documents in that category.

In [35]:
docs_per_category = dict.fromkeys(categories,0)
for category in categories:
    docs_per_category[category] = np.sum(ytrain == category)

In [36]:
grader.check("q2")

q2 results: All test cases passed!

# 3. Bag-of-words representation

Our Naive Bayes model will operate on a bag-of-words representation of each document. A bag-of-words representation is simply a set with all of the individual words of the document that are also in the vocabulary. The first task is to write the `to_bow` method, which converts a document into its bag-of-words representation.

The arguments for this method are the document as a string and the vocabulary. It should return a set (`bow`) with the unique words that appear in both the document and the vocabulary. The comments in the method provide steps to follow.

**Hints**
+ The `set` constructor
+ The `add` method on sets. 

In [48]:
def to_bow(doc,vocabulary):
    bow = set()
    
    # Split `doc` at spaces using the the string's `split` method. Obtain a list of strings.
    words_list = doc.split()

    # Keep only unique words from the list, by casting it as a set.
    word_set = set(words_list)

    # From that set, store in bow only the ones that are present in the vocabulary.
    bow = word_set.intersection(vocabulary)

    return bow

In [49]:
# This code runs 'to_bow' on each of the documents in Xtrain. 
# It produces Xtrain_bow and bag_sizes

# DO NOT MODIFY THIS CODE ...................
Xtrain_bow = np.empty(len(Xtrain),dtype=set)
bag_sizes = np.empty(len(Xtrain),dtype=int)
for i, doc in enumerate(Xtrain):
    bow = to_bow(doc,vocabulary)
    Xtrain_bow[i] = bow
    bag_sizes[i] = len(bow)
# ...........................................

In [50]:
grader.check("q3")

q3 results: All test cases passed!

# 4. Compute the document count for each word in each category
To estimate the parameters of the Naive Bayes model, we will need to know, for each category and each word, the number of documents of the category that contain the word. This is the purpose of the `find_doc_counts_per_word_category` function.

Implement the `find_doc_counts_per_word_category` following the steps provided in the code. This function accepts training data (`Xtrain_bow` and ` ytrain`), as well as the categories and the vocabulary. It produces `doc_counters`, which is a dictionary indexed by category. For each category, `doc_counters[category]` is a dictionary indexed by words in the vocabulary. For each `word` in the vocabulary, `doc_counters[category][word]` is the number of documents of that category that contain that word.

For example, if we run the `find_doc_counts_per_word_category` and save the result as `doccount_per_cat_and_word`, then we can obtain the number of documents in the computer graphics category that contain the word "windows":

```python 
doccount_per_cat_and_word = find_doc_counts_per_word_category(categories,vocabulary,ytrain,Xtrain_bow)
doccount_per_cat_and_word['comp.graphics']['windows']
```


In [53]:
def find_doc_counts_per_word_category(categories,vocabulary,ytrain,Xtrain_bow):

    # Initialize doc_counters
    doc_counters = dict.fromkeys(categories)
    for category in categories:
        doc_counters[category]  = dict.fromkeys(vocabulary,0)

    # Loop through categories.
    for category in categories:

        # Filter Xtrain_bow and keep only the documents of this category
        docs_in_category = [doc for doc, label in zip(Xtrain_bow, ytrain) if label == category]
        # For each document in this category, increment the doc_counter entry for all vocabulary words found in the document.
        for doc in docs_in_category:
            for word in doc:
                if word in vocabulary:
                    doc_counters[category][word] += 1

    return doc_counters

In [54]:
# Run `find_doc_counts_per_word_category` on every word in the training dataset

# DO NOT MODIFY THIS CODE ...................
doccount_per_cat_and_word = find_doc_counts_per_word_category(categories,vocabulary,ytrain,Xtrain_bow)
# ...........................................

In [55]:
grader.check("q4")

q4 results: All test cases passed!

# 5. Find word frequencies per category

Write the `compute_freq` method. This method takes `doccount_per_cat_and_word`, `ytrain` and the Laplace smoothing factor `alpha` as inputs and computes word and category frequencies.

+ Category frequencies `catfreq[category]`: The category frequency for category $k$ is the proportion of the documents that are of class $k$.

$$\hat{p}_k = \frac{N_k}{N} $$

where $N_k$ is the number of documents in category $k$, and $N$ is the total number of documents. 

+ Word frequencies `wordfreq[category][word]`: The Laplace-smoothed word frequency for a word $d$ and category $k$.

$$\hat{p}_{d,k} = \frac{N_{d,k}+\alpha}{N_k + \alpha D} $$

where $N_{d,k}$ is the number of documents of category $k$ that contain word $d$, and $\alpha$ is the Laplace smoothing factor. 

In [57]:
def compute_freq(doccount_per_cat_and_word,ytrain,categories,vocabulary,alpha):

    K = len(categories)  # number of categories
    D = len(vocabulary)  # number of vocabulary words
    N = len(ytrain)      # number of documents

    # Compute the number of documents in each category. Store it in the dictionary `ndocs`.
    ndocs = dict.fromkeys(categories)
    for category in categories:
        ndocs[category] = sum(1 for y in ytrain if y == category)

    # Compute the category frequenies. For each category, catfreq[category] equals
    # the number of documents of that category (ndocs) divided by the total number of documents.
    catfreq = dict()
    for category, n in ndocs.items():
        catfreq[category] = n/N

    # Initialize wordfreq
    wordfreq = dict.fromkeys(categories)
    for category in categories:
        wordfreq[category] = dict.fromkeys(vocabulary)

    # Compute wordfreq
    # For each category ...
    for category in categories:

        # the denominator is the number of documents in that category + alpha*K
        den = ndocs[category] + alpha * D

        # iterate through items in `doccount_per_cat_and_word` to compute
        # the word frequency for every category and word.
        for word, doccount in doccount_per_cat_and_word[category].items():
            wordfreq[category][word] = (doccount + alpha) / den

    return wordfreq, catfreq

In [58]:
# Run `compute_word_log_freq` with alpha=0.01.

# DO NOT MODIFY THIS CODE ...................
wordfreq, catfreq = compute_freq(doccount_per_cat_and_word,ytrain,categories,vocabulary,0.01)
# ...........................................

In [59]:
grader.check("q5")

q5 results: All test cases passed!

# 6. Write the Naive Bayes prediction function.

Use your Naive Bayes model to predict the category of a given test document `doc`. 

Recall that Naive Bayes selects the category as follows:

$$\hat{y} = \underset{k}{\text{argmax}} \hspace{2mm} \log \hat{p}_k +   \sum_{d:x^d=1}  \log \hat{p}_{d,k} +\sum_{d:x^d=0}  \log (1-\hat{p}_{d,k})$$

The arguments of the `predict` method are:
+ `doc`: a single document as a string.
+ `wordfreq`, `catfreq`: the ratios computed in the previous step (with $\alpha=0.01$)
+ `vocabulary`: the vocabulary.

The steps are:
1. Find the BOW representation of `doc`.

2. Loop through categories, for each one compute its score using the above formula.

3. Return the category with the highest score.

In [68]:
import math

def predict(doc, wordfreq, catfreq, vocabulary):
    # 1. Find the BOW representation of doc.
    doc_bow = to_bow(doc, vocabulary)
    
    # 2. Loop through categories, for each one compute its score, and save it in score_cat.
    score_cat = dict.fromkeys(catfreq.keys(), 0)
    epsilon = 1e-10  # Small value to prevent log(0)
    
    for category in catfreq.keys():
        # Start with the log of the category frequency
        score = math.log(catfreq[category] + epsilon)
        
        for word in vocabulary:
            p_dk = wordfreq[category][word]
            # Ensure probabilities are within (epsilon, 1 - epsilon)
            p_dk = min(max(p_dk, epsilon), 1 - epsilon)
            
            if word in doc_bow:
                # Word is present in document
                score += math.log(p_dk)
            else:
                # Word is not present in document
                score += math.log(1 - p_dk)
        
        score_cat[category] = score
    
    # 3. Find the category with the highest score.
    maxcat = max(score_cat, key=score_cat.get)
    
    return maxcat

In [69]:
# Run `predict` on all of the documents in the validation set.

# DO NOT MODIFY THIS CODE ...................
allpred = list()
for x in Xval:
    allpred.append( predict(x, wordfreq, catfreq, vocabulary) )
# ...........................................

allpred[0]

'rec.motorcycles'

In [70]:
grader.check("q6")

q6 results: All test cases passed!

# 7. Compute the accuracy of the model

The function `compute_accuracy` takes a dataset (`X`,`y`), computes predictions for the documents in `X` using the `predict` function, and computes the accuracy of these predictions with respect to `y`. The accuracy of the model is defined as the number of correct predictions, divided by the total number of predictions. Implement this function. 

In [75]:
def compute_accuracy(X, y, wordfreq, catfreq, vocabulary):

    # count the number of correct predictions
    correct = 0
    total_predictions = len(X)
    for i in range(len(X)):
        # Predict the category for the i-th document
        predicted_category = predict(X[i], wordfreq, catfreq, vocabulary)
        
        # Compare with the true label
        if predicted_category == y[i]:
            correct += 1

    # Accuracy is the ratio of correct predictions to total predictions
    accuracy = correct / total_predictions

    return accuracy

In [76]:
compute_accuracy(Xtrain, ytrain, wordfreq, catfreq, vocabulary)

1.0

In [77]:
grader.check("q7")

q7 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Make sure you submit the .zip file to Gradescope.

In [78]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)